In [32]:
!pip3 install pandas numpy scikit-learn fasterrisk imblearn xgboost networkx interpret sdv


  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
  Using cached torch-2.5.1-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 3.5 MB/s eta 0:00:00a 0:00:01
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl (172 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.1 MB/s eta 0:00:00a 0:00:01
Using cached torch-2.5.1-cp39-none-macosx_11_0_arm64.whl (63.9 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (18

In [ ]:
# tested on python 3.9.6

In [ ]:
import pandas as pd
import numpy as np


#import sklearn as skl
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

import fasterrisk.fasterrisk
from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier
from fasterrisk.binarization_util import convert_continuous_df_to_binary_df


/Users/berger/Documents/soccer/commentary_supplement/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
import imblearn
from imblearn.over_sampling import RandomOverSampler, SMOTENC

In [4]:
#import xgboost as xgb
#import networkx as nx


In [5]:
#import interpret
from interpret import show
from interpret.glassbox import ExplainableBoostingClassifier

# load all data and iterate over test set player IDs

The goal is to test fasterrisk and EBMs on the same test-splits like the Hecksteden et al. (2022) paper.

In [18]:
all_data=pd.read_csv("/Users/berger/Documents/soccer/positron_project/data_ml_praevent.csv")


# fasterrisk with sparsity constraint

In [ ]:
auc_frisk=[]

for i in range(1, 5):
    filename = f"data_test_{i}_old.csv"

    file = pd.read_csv(filename)
    ids=file['ID']

    ID=all_data['ID']

    y=all_data['Crit']

    binarized, featureIndex_to_groupIndex=convert_continuous_df_to_binary_df(all_data.drop(columns=["Crit",'ID']),max_num_thresholds_per_feature=25,get_featureIndex_to_groupIndex=True)

    binarized['ID']=ID
    binarized['y']=y

    #ensure player IDs from the test data are not in the train data
    train=binarized[~binarized['ID'].isin(ids)]
    test=binarized[binarized['ID'].isin(ids)]

    train=train.drop(columns=["ID"])
    test=test.drop(columns=["ID"])

    X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
    X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1]


    y_train_frisk=np.asarray(y_train)
    y_train_frisk[y_train_frisk == 1] = 1
    y_train_frisk[y_train_frisk == 0] = -1
    y_train_frisk

    y_test_frisk=np.asarray(y_test)
    y_test_frisk[y_test_frisk == 1] = 1
    y_test_frisk[y_test_frisk == 0] = -1
    y_test_frisk


    sparsity = 8
    parent_size = 10
    ub=10
    lb=ub*-1
    sparsity_constraint=8

    RiskScoreOptimizer_m = RiskScoreOptimizer(X = np.array(X_train), y = y_train_frisk, 
                                                    k = sparsity,
                                                    parent_size=parent_size, 
                                                    select_top_m=1,
                                                    ub=ub,
                                                    lb=lb,
                                                    maxAttempts=10,
                                                    featureIndex_to_groupIndex = featureIndex_to_groupIndex,
                                                    group_sparsity=sparsity_constraint)
    RiskScoreOptimizer_m.optimize()
    multipliers, sparseDiversePool_beta0_integer, sparseDiversePool_betas_integer = RiskScoreOptimizer_m.get_models()
    model_index = 0 # first model
    multiplier = multipliers[model_index]
    intercept = sparseDiversePool_beta0_integer[model_index]
    coefficients = sparseDiversePool_betas_integer[model_index]
    #RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients)

    feature_names=list(X_train.columns)
    RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients, X_train = X_train)
    #RiskScoreClassifier_m.reset_featureNames(feature_names)
    #RiskScoreClassifier_m.print_model_card()
        
    y_test_pred=RiskScoreClassifier_m.predict_prob(np.asarray(X_test))

    test_acc, test_auc = RiskScoreClassifier_m.get_acc_and_auc(np.asarray(X_test), y_test_frisk)

    fpr, tpr, thresholds = roc_curve(y_true=y_test,y_score=y_test_pred)    
    df = pd.DataFrame(zip(fpr,tpr,thresholds), columns = ['false_positive_rate', 'true_positive_rate', 'thresholds'])   
    df.to_csv("roc_curve_frisk_"+str(i)+".csv")

    auc_frisk.append(test_auc)


Converting continuous features to binary features in the dataframe......
We select thresholds for each continuous feature by sampling (without replacement) <= max_num_thresholds_per_feature values from all unique values in that feature column.
Finish converting continuous features to binary features......
Converting continuous features to binary features in the dataframe......
We select thresholds for each continuous feature by sampling (without replacement) <= max_num_thresholds_per_feature values from all unique values in that feature column.
Finish converting continuous features to binary features......
Converting continuous features to binary features in the dataframe......
We select thresholds for each continuous feature by sampling (without replacement) <= max_num_thresholds_per_feature values from all unique values in that feature column.
Finish converting continuous features to binary features......
Converting continuous features to binary features in the dataframe......
We sel

In [133]:
auc_frisk

[np.float64(0.6168531006743642),
 np.float64(0.6176590909090909),
 np.float64(0.6400319656874647),
 np.float64(0.6892719947531154)]

In [14]:
print(np.mean(auc_frisk))
print(np.std(auc_frisk))

0.6409540380060088
0.02940656772766909


# Visualize fasterrisk model

In [131]:
filename = f"data_test_{4}_old.csv"

file = pd.read_csv(filename)
ids=file['ID']

#train=all_data[~all_data['ID'].isin(ids)]
#test=all_data[all_data['ID'].isin(ids)]

ID=all_data['ID']

y=all_data['Crit']

binarized, featureIndex_to_groupIndex=convert_continuous_df_to_binary_df(all_data.drop(columns=["Crit",'ID']),max_num_thresholds_per_feature=25,get_featureIndex_to_groupIndex=True)

binarized['ID']=ID
binarized['y']=y

train=binarized[~binarized['ID'].isin(ids)]
test=binarized[binarized['ID'].isin(ids)]

train=train.drop(columns=["ID"])
test=test.drop(columns=["ID"])

X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1]


y_train_frisk=np.asarray(y_train)
y_train_frisk[y_train_frisk == 1] = 1
y_train_frisk[y_train_frisk == 0] = -1
y_train_frisk

y_test_frisk=np.asarray(y_test)
y_test_frisk[y_test_frisk == 1] = 1
y_test_frisk[y_test_frisk == 0] = -1
y_test_frisk

sparsity = 8
parent_size = 10
ub=10
lb=ub*-1
sparsity_constraint=8

RiskScoreOptimizer_m = RiskScoreOptimizer(X = np.asarray(X_train), y = y_train_frisk, 
                                                  k = sparsity,
                                                  parent_size=parent_size, 
                                                  select_top_m=10,
                                                  ub=ub,
                                                  lb=lb,
                                                  maxAttempts=10,
                                                  group_sparsity=sparsity_constraint,
                                                  featureIndex_to_groupIndex = featureIndex_to_groupIndex
                                                  )
RiskScoreOptimizer_m.optimize()
multipliers, sparseDiversePool_beta0_integer, sparseDiversePool_betas_integer = RiskScoreOptimizer_m.get_models()
model_index = 0 # first model
multiplier = multipliers[model_index]
intercept = sparseDiversePool_beta0_integer[model_index]
coefficients = sparseDiversePool_betas_integer[model_index]
#RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients)

feature_names=list(X_train.columns)
RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients, X_train = X_train)
RiskScoreClassifier_m.reset_featureNames(feature_names)
RiskScoreClassifier_m.print_model_card()



Converting continuous features to binary features in the dataframe......
We select thresholds for each continuous feature by sampling (without replacement) <= max_num_thresholds_per_feature values from all unique values in that feature column.
Finish converting continuous features to binary features......
The Risk Score is:
1.    VV_resid_age<=-96.4526726784976     -1 point(s) |   ...
2.                          IAT<=13.3     -1 point(s) | + ...
3.                    Sprint_30<=4.59     -2 point(s) | + ...
4.                 SIMS_score<=19.875     -1 point(s) | + ...
5.                 SIMS_score<=20.625      2 point(s) | + ...
6.                       SIMS_pain<=0     -1 point(s) | + ...
7.                       Matchday<=ja      1 point(s) | + ...
8.                  Srpe_team_avg<=80     -2 point(s) | + ...
                                                SCORE | =    
SCORE |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |  

In [130]:
# get Rashomon set and select 
# Model 2 selected by AH
# 
num_models = min(10, len(multipliers))

for model_index in range(num_models):
    multiplier = multipliers[model_index]
    intercept = sparseDiversePool_beta0_integer[model_index]
    coefficients = sparseDiversePool_betas_integer[model_index]

    feature_names=list(X_train.columns)
    RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients, X_train = X_train)
    RiskScoreClassifier_m.reset_featureNames(feature_names)
    RiskScoreClassifier_m.print_model_card()

The Risk Score is:
1.     VV_resid_age<=59.1942130596619     -2 point(s) |   ...
2.     VV_resid_age<=89.9301129761037      1 point(s) | + ...
3.                          Fat<=20.1     -4 point(s) | + ...
4.                 SIMS_score<=20.625      2 point(s) | + ...
5.                       SIMS_pain<=0     -3 point(s) | + ...
6. Srpe_7d_robust<=-0.457326892109501      3 point(s) | + ...
7.  Srpe_7d_robust<=0.123980424143556     -3 point(s) | + ...
8.                 Srpe_team_avg<=170     -3 point(s) | + ...
                                                SCORE | =    
SCORE |  -15.0  |  -14.0  |  -13.0  |  -12.0  |  -11.0  |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.1% |   0.2% |   0.4% |   0.7% |
SCORE |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |   4.0  |   5.0  |   6.0  |
RISK  |   1.2% |   2.1% |   3.5% |   6.0% |  10.0% |  16.1% |  25.0% |  36.6% |  50.0% |  63.4% 

# EBM loop over test sets

In [8]:
auc_ebm=[]

for i in range(1, 5):
    filename = f"data_test_{i}_old.csv"

    file = pd.read_csv(filename)
    ids=file['ID']

    train=all_data[~all_data['ID'].isin(ids)]
    test=all_data[all_data['ID'].isin(ids)]

    test=test[train.columns]

    test.dtypes==train.dtypes

    del test[test.columns[0]]
    del train[train.columns[0]]


    numeric_columns = ['Age', 'Fat','IAT','Sprint_30','SIMS_score','Srpe_7d_robust','Srpe_team_avg','KEB_AB_robust']
    train[numeric_columns] = train[numeric_columns].apply(pd.to_numeric, errors='coerce')

    categorical_columns = ['After_RTP', 'Pos_code','SIMS_pain','Matchday','Crit']
    train[categorical_columns] = train[categorical_columns].astype('category')

    numeric_columns = ['Age', 'Fat','IAT','Sprint_30','SIMS_score','Srpe_7d_robust','Srpe_team_avg','KEB_AB_robust']
    test[numeric_columns] = test[numeric_columns].apply(pd.to_numeric, errors='coerce')

    categorical_columns = ['After_RTP', 'Pos_code','SIMS_pain','Matchday','Crit']
    test[categorical_columns] = test[categorical_columns].astype('category')

    X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
    X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1]

    ebm=ExplainableBoostingClassifier(
            inner_bags=10,
            interactions=0,
            learning_rate=0.02,
            max_bins=20,
            max_interaction_bins=16,
            max_rounds=20,
            outer_bags=12,
            validation_size=0.2,
            n_jobs=-2,
            random_state=12)
    
    ebm.fit(X_train[['Srpe_team_avg','Srpe_7d_robust','Pos_code','Sprint_30']],y_train)
    test_auc=roc_auc_score(y_true=y_test,y_score=ebm.predict_proba(X_test)[:,1])
    
    fpr, tpr, thresholds = roc_curve(y_true=y_test,y_score=ebm.predict_proba(X_test)[:,1])    
    df = pd.DataFrame(zip(fpr,tpr,thresholds), columns = ['false_positive_rate', 'true_positive_rate', 'thresholds'])   
    df.to_csv("roc_curve_ebm_"+str(i)+".csv")


    auc_ebm.append(test_auc)

/var/folders/0_/3hlfdkd50633tgz9cbkx_vcw18f2ds/T/ipykernel_15072/3930493632.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[numeric_columns] = train[numeric_columns].apply(pd.to_numeric, errors='coerce')
/var/folders/0_/3hlfdkd50633tgz9cbkx_vcw18f2ds/T/ipykernel_15072/3930493632.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[categorical_columns] = train[categorical_columns].astype('category')
/var/folders/0_/3hlfdkd50633tgz9cbkx_vcw18f2ds/T/ipykernel_15072/3930493632.py:21: SettingWit

In [14]:
print(np.mean(auc_ebm))
print(np.std(auc_ebm))

0.6113775084022712
0.05439086384134627


# Visualize EBM

In [15]:
filename = f"data_test_{1}_old.csv"

file = pd.read_csv(filename)
ids=file['ID']

train=all_data[~all_data['ID'].isin(ids)]
test=all_data[all_data['ID'].isin(ids)]

test=test[train.columns]

test.dtypes==train.dtypes

del test[test.columns[0]]
del train[train.columns[0]]


numeric_columns = ['Age', 'Fat','IAT','Sprint_30','SIMS_score','Srpe_7d_robust','Srpe_team_avg','KEB_AB_robust']
train[numeric_columns] = train[numeric_columns].apply(pd.to_numeric, errors='coerce')

categorical_columns = ['After_RTP', 'Pos_code','SIMS_pain','Matchday','Crit']
train[categorical_columns] = train[categorical_columns].astype('category')

numeric_columns = ['Age', 'Fat','IAT','Sprint_30','SIMS_score','Srpe_7d_robust','Srpe_team_avg','KEB_AB_robust']
test[numeric_columns] = test[numeric_columns].apply(pd.to_numeric, errors='coerce')

categorical_columns = ['After_RTP', 'Pos_code','SIMS_pain','Matchday','Crit']
test[categorical_columns] = test[categorical_columns].astype('category')

X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1]

ebm=ExplainableBoostingClassifier(
            inner_bags=10,
            interactions=0,
            learning_rate=0.02,
            max_bins=20,
            max_interaction_bins=16,
            max_rounds=20,
            outer_bags=12,
            validation_size=0.2,
            n_jobs=-2,
            random_state=12)
ebm.fit(X_train[['Srpe_team_avg','Srpe_7d_robust','Pos_code','Sprint_30']],y_train)


show(ebm.explain_global())

/var/folders/0_/3hlfdkd50633tgz9cbkx_vcw18f2ds/T/ipykernel_82731/1983080263.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[numeric_columns] = train[numeric_columns].apply(pd.to_numeric, errors='coerce')
/var/folders/0_/3hlfdkd50633tgz9cbkx_vcw18f2ds/T/ipykernel_82731/1983080263.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[categorical_columns] = train[categorical_columns].astype('category')


<!-- http://127.0.0.1:7001/5041704144/ -->

In [255]:
# extract data to plot using ggplot instead of visualization provided by ebm.explain_global()

ebm_global=ebm.explain_global()
feature_data = []
#Loop through all features (numerical and categorical)
for i in range(len(ebm_global.data()['names'])):
    data_names = ebm_global.data()
    feature_name = data_names['names'][i]
    shape_data = ebm_global.data(i)

    # Check if the feature is not an interaction
    if shape_data['type'] == 'univariate':
        
        # Detect categorical features if names or values are strings
        if isinstance(shape_data['names'][0], str):
            # Extract categories and corresponding scores
            categories = shape_data['names']
            scores = shape_data['scores']
            upper_bounds = shape_data['upper_bounds']
            lower_bounds = shape_data['lower_bounds']
            
            # Loop through and store the category data
            for category, score, upper_bounds, lower_bounds in zip(categories, scores,upper_bounds, lower_bounds):
                feature_data.append({
                    'Feature': feature_name,
                    'Type': 'Categorical',
                    'Category': category,
                    'Score': score,
                    'upper':upper_bounds,
                    'lower':lower_bounds
                })
        
        # Detect numerical features if names or values are floats or integers
        elif isinstance(shape_data['names'][0], (int, float)):
            # Extract bin edges and corresponding scores
            x_values = shape_data['names']
            scores = shape_data['scores']
            upper_bounds = shape_data['upper_bounds']
            lower_bounds = shape_data['lower_bounds']
            
            # Loop through and store the bin data
            for x_values, score, upper_bounds, lower_bounds in zip(x_values, scores,upper_bounds,lower_bounds):
                feature_data.append({
                    'Feature': feature_name,
                    'Type': 'Numerical',
                    'Value': x_values,
                    'Score': score,
                    'upper':upper_bounds,
                    'lower':lower_bounds
                })

            score_range = shape_data['names']
            max_range_value = max(score_range)
            second_last_score = scores[-1]  # Last score or second-to-last value
            
            # Add the new max range bin
            feature_data.append({
                'Feature': feature_name,
                'Type': 'Numerical',
                'Value': max_range_value,
                'Score': second_last_score,  # Use the second-to-last score
                'upper': upper_bounds,  # Keep the same upper bound as the last bin
                'lower': lower_bounds   # Keep the same lower bound as the last bin
            })

# Convert the list of dictionaries into a pandas DataFrame
feature_df = pd.DataFrame(feature_data)
feature_df.to_csv("feature_data.csv", index=False)
print(feature_df)

          Feature       Type    Value     Score     upper     lower Category
0   Srpe_team_avg  Numerical    0.000 -0.004466 -0.003770 -0.005161      NaN
1   Srpe_team_avg  Numerical    1.000 -0.004137 -0.003462 -0.004812      NaN
2   Srpe_team_avg  Numerical   48.500 -0.003506 -0.002779 -0.004234      NaN
3   Srpe_team_avg  Numerical  103.500 -0.002951 -0.002224 -0.003679      NaN
4   Srpe_team_avg  Numerical  181.000 -0.002351 -0.001644 -0.003059      NaN
..            ...        ...      ...       ...       ...       ...      ...
56      Sprint_30  Numerical    4.185  0.000870  0.001581  0.000160      NaN
57      Sprint_30  Numerical    4.205  0.000852  0.001493  0.000211      NaN
58      Sprint_30  Numerical    4.245  0.001643  0.002363  0.000924      NaN
59      Sprint_30  Numerical    4.325  0.001823  0.002665  0.000981      NaN
60      Sprint_30  Numerical    5.620  0.001823  0.002665  0.000981      NaN

[61 rows x 7 columns]


# Note at this point: switch to R code to see ggplot visualization. The script is called plots.R

# Use synthetic data to show that the code works

Here, we will be using synthetic data generated using the original data used in Hecksteden et al. (2022). This is purely to show that the code works, in addition to the code above used to generate orginal results presented in the commentary.

## generate synthetic data


In [35]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata

In [ ]:
all_data=pd.read_csv("/Users/berger/Documents/soccer/positron_project/data_ml_praevent.csv")
len(np.unique(all_data['ID']))

In [ ]:
#generate 100 rows for every player in original data

synth=pd.DataFrame()
for iter, player in enumerate(np.unique(all_data['ID'])):
    
    #player=105
    temp=all_data[all_data['ID']==player]

    #y=temp['Crit']
    temp=temp.drop(columns=['ID'])

    synthesizer = GaussianCopulaSynthesizer(Metadata.detect_from_dataframe(temp), enforce_min_max_values=False, enforce_rounding=False)
    synthesizer.fit(data=temp)

    synthetic_data = synthesizer.sample(num_rows=100)
    synthetic_data

    synth=synth.append(synthetic_data)

synth.to_csv("synthetic_data.csv")

## fasterrisk on synthetic data

In [75]:
synth=pd.read_csv("synthetic_data.csv")

In [76]:
y=synth['Crit']
X=synth.drop(columns=["Crit","Unnamed: 0"])
X

,After_RTP,Age,Pos_code,VV_resid_age,Fat,IAT,Sprint_30,SIMS_score,SIMS_pain,Srpe_7d_robust,Matchday,Srpe_team_avg,KEB_AB_robust
0,0,29,0,149.335755,12.55972,12.996188,4.210000,13.485,0,-0.846467,nein,243,0
1,0,28,0,175.930113,14.40000,13.200000,4.336528,13.875,0,-0.830450,nein,359,-1
2,0,28,0,175.930113,14.40000,13.200000,4.336528,13.875,0,1.259789,ja,481,0
3,0,28,0,175.930113,14.40000,13.200000,4.336528,13.875,0,-1.352950,nein,59,-1
4,0,28,0,175.930113,14.40000,13.200000,4.336528,13.875,0,1.115926,nein,380,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,1,19,0,-20.530232,15.00000,14.800000,4.030000,19.875,0,-0.748555,ja,475,0
8796,0,19,0,-30.965315,15.00000,14.800000,4.030000,19.875,0,-1.416780,nein,363,0
8797,1,19,0,-18.046713,15.00000,14.800000,4.030000,19.875,0,-0.707965,nein,97,0
8798,0,19,0,-9.991478,15.00000,14.800000,4.030000,19.875,0,-0.023238,nein,470,1


In [77]:
binarized, featureIndex_to_groupIndex=convert_continuous_df_to_binary_df(X,max_num_thresholds_per_feature=25,get_featureIndex_to_groupIndex=True)
binarized

X_train, X_test, y_train, y_test=train_test_split(binarized,y,test_size=.2,random_state=42)

y_train_frisk=np.asarray(y_train)
y_train_frisk[y_train_frisk == 1] = 1
y_train_frisk[y_train_frisk == 0] = -1
y_train_frisk
y_test_frisk=np.asarray(y_test)
y_test_frisk[y_test_frisk == 1] = 1
y_test_frisk[y_test_frisk == 0] = -1
y_test_frisk


sparsity = 8
parent_size = 10
ub=10
lb=ub*-1
sparsity_constraint=8

RiskScoreOptimizer_m = RiskScoreOptimizer(X = np.array(X_train), y = y_train_frisk, 
    k = sparsity,
    parent_size=parent_size, 
    select_top_m=1,
    ub=ub,
    lb=lb,
    maxAttempts=10,
    featureIndex_to_groupIndex = featureIndex_to_groupIndex,
    group_sparsity=sparsity_constraint)



RiskScoreOptimizer_m.optimize()
multipliers, sparseDiversePool_beta0_integer, sparseDiversePool_betas_integer = RiskScoreOptimizer_m.get_models()
model_index = 0 # first model
multiplier = multipliers[model_index]
intercept = sparseDiversePool_beta0_integer[model_index]
coefficients = sparseDiversePool_betas_integer[model_index]
#RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients)

feature_names=list(X_train.columns)
RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients, X_train = X_train)
#RiskScoreClassifier_m.reset_featureNames(feature_names)
#RiskScoreClassifier_m.print_model_card()

y_test_pred=RiskScoreClassifier_m.predict_prob(np.asarray(X_test))

test_acc, test_auc = RiskScoreClassifier_m.get_acc_and_auc(np.asarray(X_test), y_test_frisk)



Converting continuous features to binary features in the dataframe......
We select thresholds for each continuous feature by sampling (without replacement) <= max_num_thresholds_per_feature values from all unique values in that feature column.
Finish converting continuous features to binary features......


In [78]:
print(test_auc)

0.5909026195899773


In [79]:

feature_names=list(binarized.columns)
RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients, X_train = X_train)
RiskScoreClassifier_m.reset_featureNames(feature_names)
RiskScoreClassifier_m.print_model_card()

The Risk Score is:
1.              Fat<=9.005563726609063     -4 point(s) |   ...
2.               Fat<=9.33690095015295      7 point(s) | + ...
3.             IAT<=13.644931546399306     -2 point(s) | + ...
4.        Sprint_30<=4.019416735365593     -2 point(s) | + ...
5.                    SIMS_score<=21.5      5 point(s) | + ...
6. Srpe_7d_robust<=-0.8503627156519471      3 point(s) | + ...
7.  Srpe_7d_robust<=0.3092389238165474      2 point(s) | + ...
8.                  Srpe_team_avg<=131     -2 point(s) | + ...
                                                 SCORE | =    
SCORE |  -10.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |
SCORE |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  17.0  |
RISK  |   0.0% |   0.1% |   0.1% |   0

## EBM on synthetic data

In [80]:

numeric_columns = ['Age', 'Fat','IAT','Sprint_30','SIMS_score','Srpe_7d_robust','Srpe_team_avg','KEB_AB_robust']
X[numeric_columns] = X[numeric_columns].apply(pd.to_numeric, errors='coerce')

categorical_columns = ['After_RTP', 'Pos_code','SIMS_pain','Matchday']
X[categorical_columns] = X[categorical_columns].astype('category')

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2,random_state=42)


ebm=ExplainableBoostingClassifier(
        inner_bags=10,
        interactions=0,
        learning_rate=0.02,
        max_bins=20,
        max_interaction_bins=16,
        max_rounds=20,
        outer_bags=12,
        validation_size=0.2,
        n_jobs=-2,
        random_state=12)

ebm.fit(X_train[['Srpe_team_avg','Srpe_7d_robust','Pos_code','Sprint_30']],y_train)
test_auc=roc_auc_score(y_true=y_test,y_score=ebm.predict_proba(X_test)[:,1])


In [81]:
print(test_auc)

0.5410734624145785


In [82]:

show(ebm.explain_global())

<!-- http://127.0.0.1:7001/14492857776/ -->